In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop unused columns
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [3]:
# Bin rarities
app_type_counts = application_df.APPLICATION_TYPE.value_counts()
replace_application = app_type_counts[app_type_counts <= 250].index
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

app_class_counts = application_df.CLASSIFICATION.value_counts()
replace_class = app_class_counts[app_class_counts <= 1500].index
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [4]:
# OPTIMIZATION 1: Remove noisy column(s)
# Categorize ASK_AMT column and bin rarities
bins = [(i*1000)**1.5 for i in range(20)]
application_df['ASK_AMT_BIN'] = pd.cut(application_df.ASK_AMT, bins).astype(str)
application_df.ASK_AMT_BIN = application_df.ASK_AMT_BIN.replace('nan', 'Other')
application_df.ASK_AMT_BIN.value_counts()

# Remove noisy column
application_df.drop(columns=['ASK_AMT'], inplace=True)

In [5]:
# Encode categorical columns
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.to_list()

enc = OneHotEncoder(sparse=False)
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat].values))
encode_df.columns = enc.get_feature_names(application_cat)
application_df_encoded = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)

C:\Users\lself\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\lself\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


In [6]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop('IS_SUCCESSFUL',1)
y = application_df_encoded.IS_SUCCESSFUL

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\lself\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 =  5
hidden_nodes_layer2 = 4

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 315       
                                                                 
 dense_1 (Dense)             (None, 4)                 24        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 344
Trainable params: 344
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('AlphabetSoupCharity_Optimization_1.h5', save_weights_only=True, save_freq=5)
fit_model = nn1.fit(X_train_scaled, y_train, epochs=100, callbacks=[mc])

Epoch 1/100
804/804 [==============================] - 11s 13ms/step - loss: 0.6148 - accuracy: 0.6859
Epoch 2/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5739 - accuracy: 0.7145
Epoch 3/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5674 - accuracy: 0.7171
Epoch 4/100
804/804 [==============================] - 9s 12ms/step - loss: 0.5629 - accuracy: 0.7200
Epoch 5/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5614 - accuracy: 0.7211
Epoch 6/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5605 - accuracy: 0.7207
Epoch 7/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5592 - accuracy: 0.7215
Epoch 8/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5588 - accuracy: 0.7213
Epoch 9/100
804/804 [==============================] - 9s 11ms/step - loss: 0.5582 - accuracy: 0.7226
Epoch 10/100
804/804 [==============================] - 9s 11ms/step - loss

804/804 [==============================] - 12s 15ms/step - loss: 0.5502 - accuracy: 0.7259
Epoch 81/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5508 - accuracy: 0.7248
Epoch 82/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5503 - accuracy: 0.7255
Epoch 83/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5503 - accuracy: 0.7249
Epoch 84/100
804/804 [==============================] - 9s 12ms/step - loss: 0.5503 - accuracy: 0.7257
Epoch 85/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5502 - accuracy: 0.7255
Epoch 86/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5503 - accuracy: 0.7266
Epoch 87/100
804/804 [==============================] - 9s 12ms/step - loss: 0.5504 - accuracy: 0.7257
Epoch 88/100
804/804 [==============================] - 9s 11ms/step - loss: 0.5503 - accuracy: 0.7257
Epoch 89/100
804/804 [==============================] - 10s 12ms/step - loss: 0.

In [9]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5536 - accuracy: 0.7298 - 515ms/epoch - 2ms/step
Loss: 0.5535944104194641, Accuracy: 0.7297959327697754


In [10]:
# OPTIMIZATION 2: Add neurons - neuron counts were doubled for both hidden layers
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 6

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                630       
                                                                 
 dense_4 (Dense)             (None, 6)                 66        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 703
Trainable params: 703
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('AlphabetSoupCharity_Optimization_2.h5', save_weights_only=True, save_freq=5)
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100, callbacks=[mc])

Epoch 1/100
804/804 [==============================] - 12s 15ms/step - loss: 0.6240 - accuracy: 0.6752
Epoch 2/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5733 - accuracy: 0.7185
Epoch 3/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5624 - accuracy: 0.7235
Epoch 4/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5587 - accuracy: 0.7249
Epoch 5/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5567 - accuracy: 0.7254
Epoch 6/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5548 - accuracy: 0.7269
Epoch 7/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5539 - accuracy: 0.7271
Epoch 8/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5530 - accuracy: 0.7278
Epoch 9/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5520 - accuracy: 0.7274
Epoch 10/100
804/804 [==============================] - 11s 13ms/step - l

804/804 [==============================] - 11s 13ms/step - loss: 0.5412 - accuracy: 0.7352
Epoch 80/100
804/804 [==============================] - 12s 15ms/step - loss: 0.5413 - accuracy: 0.7351
Epoch 81/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5413 - accuracy: 0.7345
Epoch 82/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5415 - accuracy: 0.7355
Epoch 83/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5411 - accuracy: 0.7355
Epoch 84/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5413 - accuracy: 0.7351
Epoch 85/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5409 - accuracy: 0.7356
Epoch 86/100
804/804 [==============================] - 12s 14ms/step - loss: 0.5409 - accuracy: 0.7343
Epoch 87/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5411 - accuracy: 0.7349
Epoch 88/100
804/804 [==============================] - 10s 13ms/step - loss:

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5529 - accuracy: 0.7347 - 471ms/epoch - 2ms/step
Loss: 0.5528742074966431, Accuracy: 0.7346938848495483


In [13]:
# OPTIMIZATION 3: Changing activation function - activation for first layer was changed to TanH
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 6

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                630       
                                                                 
 dense_7 (Dense)             (None, 6)                 66        
                                                                 
 dense_8 (Dense)             (None, 1)                 7         
                                                                 
Total params: 703
Trainable params: 703
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('AlphabetSoupCharity_Optimization_3.h5', save_weights_only=True, save_freq=5)
fit_model = nn3.fit(X_train_scaled, y_train, epochs=100, callbacks=[mc])

Epoch 1/100
804/804 [==============================] - 20s 25ms/step - loss: 0.5987 - accuracy: 0.6869
Epoch 2/100
804/804 [==============================] - 12s 14ms/step - loss: 0.5646 - accuracy: 0.7225
Epoch 3/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5582 - accuracy: 0.7250
Epoch 4/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5550 - accuracy: 0.7261
Epoch 5/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5528 - accuracy: 0.7276
Epoch 6/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5511 - accuracy: 0.7278
Epoch 7/100
804/804 [==============================] - 12s 14ms/step - loss: 0.5504 - accuracy: 0.7285
Epoch 8/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5494 - accuracy: 0.7295
Epoch 9/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5489 - accuracy: 0.7279
Epoch 10/100
804/804 [==============================] - 11s 13ms/step - l

804/804 [==============================] - 11s 13ms/step - loss: 0.5382 - accuracy: 0.7366
Epoch 80/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5381 - accuracy: 0.7362
Epoch 81/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5381 - accuracy: 0.7364
Epoch 82/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5378 - accuracy: 0.7369
Epoch 83/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5379 - accuracy: 0.7361
Epoch 84/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5380 - accuracy: 0.7363
Epoch 85/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5380 - accuracy: 0.7362
Epoch 86/100
804/804 [==============================] - 10s 13ms/step - loss: 0.5376 - accuracy: 0.7368
Epoch 87/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5379 - accuracy: 0.7365
Epoch 88/100
804/804 [==============================] - 11s 13ms/step - loss:

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5493 - accuracy: 0.7336 - 450ms/epoch - 2ms/step
Loss: 0.549296498298645, Accuracy: 0.7336443066596985


In [16]:
# OPTIMIZATION 4: Add hidden layers - two hidden layers were added
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 6
hidden_nodes_layer3 = 6
hidden_nodes_layer4 = 6

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 10)                630       
                                                                 
 dense_10 (Dense)            (None, 6)                 66        
                                                                 
 dense_11 (Dense)            (None, 6)                 42        
                                                                 
 dense_12 (Dense)            (None, 6)                 42        
                                                                 
 dense_13 (Dense)            (None, 1)                 7         
                                                                 
Total params: 787
Trainable params: 787
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('AlphabetSoupCharity_Optimization_4.h5', save_weights_only=True, save_freq=5)
fit_model = nn4.fit(X_train_scaled, y_train, epochs=100, callbacks=[mc])

Epoch 1/100
804/804 [==============================] - 12s 14ms/step - loss: 0.5983 - accuracy: 0.6972
Epoch 2/100
804/804 [==============================] - 12s 15ms/step - loss: 0.5676 - accuracy: 0.7247
Epoch 3/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5597 - accuracy: 0.7262
Epoch 4/100
804/804 [==============================] - 13s 17ms/step - loss: 0.5560 - accuracy: 0.7279
Epoch 5/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5542 - accuracy: 0.7289
Epoch 6/100
804/804 [==============================] - 12s 15ms/step - loss: 0.5523 - accuracy: 0.7302
Epoch 7/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5513 - accuracy: 0.7304
Epoch 8/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5504 - accuracy: 0.7319
Epoch 9/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5501 - accuracy: 0.7319
Epoch 10/100
804/804 [==============================] - 12s 15ms/step - l

804/804 [==============================] - 11s 14ms/step - loss: 0.5379 - accuracy: 0.7393
Epoch 80/100
804/804 [==============================] - 12s 14ms/step - loss: 0.5378 - accuracy: 0.7383
Epoch 81/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5380 - accuracy: 0.7386
Epoch 82/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5379 - accuracy: 0.7393
Epoch 83/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5377 - accuracy: 0.7391
Epoch 84/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5373 - accuracy: 0.7397
Epoch 85/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5379 - accuracy: 0.7397
Epoch 86/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5376 - accuracy: 0.7389
Epoch 87/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5376 - accuracy: 0.7393
Epoch 88/100
804/804 [==============================] - 11s 14ms/step - loss:

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5535 - accuracy: 0.7338 - 501ms/epoch - 2ms/step
Loss: 0.5535131096839905, Accuracy: 0.7337609529495239
